In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

End-to-end Machine Learning

Link do dataset: https://www.kaggle.com/budincsevity/szeged-weather

O problema trata da tentativa de se determinar a Temperatura (C) do hambiente, dado alguns parametros, e em principal, e humidade.

In [ ]:
#Inicialmente o dataset é carregado e brevemente análisado

#É possível notal que há uma das colunas que possui menos linhas que outras.
#Existem várias tecnicas que permitem o preencimento desses espaços que estão em falta, 
#mas tendo em vista que essa coluna não será tão relevante assim para a análise, ela não será levada em conta.

#A titulo de exemplicação, apenas a coluna "Summary" do tipo object e todas as outras colunas do tipo float64 
#serão usadas no aprendizado.

data1 = pd.read_csv('weatherHistory.csv')
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 12 columns):
Formatted_Date              96453 non-null object
Summary                     96453 non-null object
Precip_Type                 95936 non-null object
Temperature_(C)             96453 non-null float64
Apparent_Temperature_(C)    96453 non-null float64
Humidity                    96453 non-null float64
Wind_Speed_(km/h)           96453 non-null float64
Wind_Bearing_(degrees)      96453 non-null float64
Visibility_(km)             96453 non-null float64
Loud_Cover                  96453 non-null float64
Pressure_(millibars)        96453 non-null float64
Daily_Summary               96453 non-null object
dtypes: float64(8), object(4)
memory usage: 8.8+ MB


In [ ]:
#Aqui, são plotados os histogramas das váriaveis do tipo float64

data1.hist(bins=50, figsize=(8,6))
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#Se é necessário a separação do dataset em test e treino, seguindo com 20% dos dados usados para teste
#e o restante para treino

train_set, test_set = train_test_split(data1, test_size=0.2, random_state=35)

print("data has {} instances\n {} train instances\n {} test intances".
      format(len(data1),len(train_set),len(test_set)))

In [ ]:
train = train_set.copy()

In [ ]:
#Nesse estágio é onde serão buscadas as colunas com melhor correlação com a Temperatura (C)

#Se nota uma alta correção positiva com a coluna Apparent_Temperature_(C), o que ja era esperado
# pois a sensação termina geralmente segue em torno da temperatura real

#E uma segunda boa correlação, mas dessa vez negativa com a coluna Humidity

corr_matrix = train.corr()
corr_matrix["Temperature_(C)"].\
  sort_values(ascending=False)

In [ ]:
import seaborn as sns
sns.heatmap(train.corr(), 
            annot=True, fmt=".2f")

In [ ]:
# A retirada das colunas "Formatted_Date", "Precip_Type", "Daily_Summary" foi feita com o intuito de
# simplificar e diminuição do numero de colunas que será necessário o uso do one-hot encoding, diminuido o tempo
# de execução das celulas
train_X = train_set.drop(["Temperature_(C)", "Formatted_Date", "Precip_Type", "Daily_Summary"], axis=1)

train_y = train_set["Temperature_(C)"].copy()

In [ ]:
#Teste que garante que nenhuma das colunas tem nenhuma valor nulo

train_X.isnull().sum()

In [ ]:
#A média das temperaturas e seu desvio padão, que são parametos importantes na hora de avaliar o erro 
#do treino

print('Média das temperaturas: ', train_set["Temperature_(C)"].mean())
print('Desvio Padrão das temperaturas: ', train_set["Temperature_(C)"].std())

In [ ]:
#Aqui são os valores que a variável categorica assume

train_X.Summary.value_counts()

In [ ]:
#Aqui é declarada a class que será usada no pipeline para a seleção de colunas

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.attribute_names].values

In [ ]:
#Pipeline

from sklearn.pipeline import FeatureUnion
from future_encoders import OneHotEncoder

train_X_num = train_X.drop("Summary", axis=1)

num_list = list(train_X_num)

cat_attribs = ["Summary"]

num_pipeline = Pipeline([('selector', DataFrameSelector(num_list)),
                         ('std_scaler', StandardScaler())
                        ])


cat_pipeline = Pipeline([('selector', DataFrameSelector(cat_attribs)),
                         ('cat_encoder', OneHotEncoder(sparse=False))
                        ])



test_pipeline = FeatureUnion(transformer_list=[("num_pipeline", num_pipeline),
                                               ("cat_pipeline", cat_pipeline)
                                              ])

train_X_pipeline = test_pipeline.fit_transform(train_X)

In [ ]:
from sklearn.linear_model import LinearRegression

#Criação do modelo em regreção linear
lin_reg = LinearRegression()

#Usando o fit ("fitando") o modelo
lin_reg.fit(train_X_pipeline, train_y)

In [ ]:
train_X_pipeline.shape

In [ ]:
#Predição dos valores de Temperatura

print("Predictions:", lin_reg.predict(train_X_pipeline)) 

In [ ]:
from sklearn.metrics import mean_squared_error

#Calculo do erro

temp_predictions = lin_reg.predict(train_X_pipeline)
lin_mse = mean_squared_error(train_y, temp_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse